In [1]:
# Aurthor: Elihu Essien-Thomspon
# Program Description: Experiment 2
# Finding the best Ant Colony Optimization
# Algorithm Representative to use for
# the final Experiment.

import numpy as np
import random
import math

In [2]:
# Returns the distance between two 
# points(x, y) on the map

def cityDistance(map, cityID1, cityID2):
    x = abs(map[cityID1][0] - map[cityID2][0])
    y = abs(map[cityID1][1] - map[cityID2][1])
    
    # pythagoras
    dist = math.sqrt(x**2 + y**2)
    return(dist)

In [3]:
# Reading in map data from text file
# and returning a distance matrix to use
# for calculations
def getMap(size,id):
    cities = np.empty((0,2), dtype=int)      # initialize empty 2D vector list
    
    # open file
    with open(f"C:/Users/C14460702/Dissertation/Data/Maps/Size - {size}/map{id}.txt", 'r') as f:
        # extract vector list data from the comma separated string list data
        for line in f:
            line = line.strip('\n')    # remove '\n'
            vec = line.split(', ')     # split by ', '
            
            # cast co-ordinate vector as integers and insert into list 
            cities = np.append(cities, [[ int(vec[0]), int(vec[1]) ]], axis=0)
    
    
    # initialize distance table
    mapSize = len(cities)
    distanceTable = np.zeros((mapSize,mapSize))
    for row in range(mapSize):
        for col in range(mapSize):
            distanceTable[row][col] = cityDistance(cities, row, col)
    
    # return distance matrix
    return (distanceTable)

In [4]:
# Function takes in a route list
# and a distnce matrix to be used
# to score the route given

def routeScore(route, distanceTable):
    
    dist = 0;
    
    # aggregate distances between cities
    # on the list given
    for i in range(len(route)):
        cur = route[i]
        nxt = route[(i+1) % len(route)]  # includes loop back to start
        dist += distanceTable[cur][nxt]  # get the distances from the matrix
        
    
    # the smaller the distance, the higher the score
    return (1/dist)

In [5]:
# maps a variable exising on one range to another range
def mapRanges(value, currlow, currHigh, newLow, newHigh):
    # extract ranges
    currRange = currHigh - currlow
    newRange = newHigh - newLow
    
    # percentage of range that the value takes up
    valPercent = float(value - currlow) / float(currRange)
    
    # translate percentage to new range
    newVal = (newRange * valPercent) + newLow
    
    return(newVal)

In [6]:
# This function takes in a 1D array of choices
# where the each member's ID represents the choice
# and each member's value represents the percent (0-1)
# that member has of comming true. The algorithm then
# selects a winner by performing a weighted random choice

def weightedChoice(percentages):
    choiceIDs = [i for i in range(len(percentages))]
    percentages = [mapRanges(i, 0, 1, 0, 100) for i in percentages]
    
    # choose winner
    winnerID = random.choices(choiceIDs, weights = percentages, k = 1)[0]
    return (winnerID)


In [7]:
def Ant(startingPosition, mapSize, distanceTable, pheramoneTable, alpha, beta):
    position = startingPosition
    visited = []
    
    
    
    while (len(visited) < mapSize-1):
        # add current city possition to the visited list
        visited += [position]
        
        # Step 1 Select edge from unvisited edges
        # city choices is an array[cityID, choice probability]
        cityChoices = [[i,0] for i in range(mapSize) if i not in visited]


        # calculate total sum of posibilities
        IDs = [i[0] for i in cityChoices]
        total = sum([(alpha * pheramoneTable[position][cityID]) * (beta * (1/distanceTable[position][cityID])) for cityID in IDs])
        
        
        # calculate city choice probabilities
        for row in range(len(cityChoices)):
            cityID = cityChoices[row][0]
            cityChoices[row][1] = ((alpha * pheramoneTable[position][cityID]) * (beta * (1/distanceTable[position][cityID])))/total

        try:
            # choose and use a path based on those probabilities
            percentages = [i[1] for i in cityChoices]
            position = cityChoices[ weightedChoice(percentages) ][0]
        except ValueError:
            print(f"Visited - {visited}\nChoices - {IDs}\nPercentages - {[i[1] for i in cityChoices]}\nAlpha - {alpha}\nBeta - {beta}")
            print(f"\n\n*** E({position})-E({cityID})***")
            print(f"Pheramone level - {pheramoneTable[position][cityID]}\nDistance - {distanceTable[position][cityID]}")
    
    # add the final position to the list
    visited += [position]
    
    # the order in which the cities were visited
    return(visited)

In [8]:
# Pheramone update function for the Ant System (AS)
# for the TSP allows all ants to update the pheramone
# table adding pheramones to thier traveled paths 
# relative to the overall score of thier chosen route
def AS_PheramoneUpdate(routes, scores, pheramoneTable):
    
    
    for i in range(len(routes)):
        route = routes[i]
        
        # update pheramone levels for the paths taken
        # with the overall score of the route
        for j in range(len(route)):
            cur = route[j]
            nxt = route[(j+1) % len(route)]
            
            # update each edge with the score for that route
            pheramoneTable[cur][nxt] += scores[i]
            pheramoneTable[nxt][cur] += scores[i]
    
    return (pheramoneTable)

In [9]:
# This fitness function simultaneously picks out {popSize} number of mating 
# candidates from a wheel of potential candidates. The candidates are ranked 
# based on thier evaluated scores and that rank becomes thier fitness score, 
# represented by the proportion of space they take up on a matting Pool
# selection wheel.

def RankBasedSampling(population, scores, popSize, mapSize):
    
    # sort population by score accending
    sortedPopulation = [i for _,i in sorted(zip(scores,population))]
    
    # Create roulette wheel type mating pool with fitness coresponding to the slots alloted
    wheel = np.empty((0, mapSize), dtype=int)
    
    for fitness in range(popSize):        # fitness coresponds to location in sorted the array (ID)
        for i in range(fitness+1):        # e.g. ID[0] gets 1 slot and ID[last] gets {popSize} slots
            wheel = np.append(wheel,[sortedPopulation[fitness]],axis=0)
            
    
    # interval used to simultaneously select {popSize} mating indivduals
    wheelSize = np.size(wheel,axis=0)
    interval = int(wheelSize/popSize)
    
    # spin wheel
    np.random.shuffle(wheel)
    
    matingPool = []                                 # empty mating pool
    for i in range(0, wheelSize, interval):         # evenly spaced pointer locations
        matingPool += [wheel[i].tolist()]           # populate mating pool based on pointers
        
    return(matingPool)

In [10]:
# Steady State Function takes in 4 routes
# and returns for mating only the 2 children
# scoring greatest 

# The algorithm works by splitting the 4 routes
# into 2 groups (a-b) and (c-d) and by comparing
# the winners of the groups, the best two can be
# isolated and sent back
def SteadyStateFunction(routeA,routeB,routeC,routeD, distanceTable):
    # scores
    a = routeScore(routeA,distanceTable)
    b = routeScore(routeB,distanceTable)
    c = routeScore(routeC,distanceTable)
    d = routeScore(routeD,distanceTable)
    
    
    if(a>b):                                    # a is the greatest from Group1
        if(c>d):                                  # c is the greatest from Group2
            if(d>a):                                # if the greatest from Group1 is lower than the lowest from Group2
                return(routeC,routeD)                   # Group2 wins
            else:                                   # else the greatest in group1 has won twice so it's definately being chosen
                if(b>c):                              # is the lowest in Group1 also greater than the greatest in Group2
                    return(routeA,routeB)                 # if so, Group1 wins
                else:
                    return(routeA,routeC)                 # if not, only the greatest in Group1 and in Group2 win
        else:                                     # Same logic as above but the greatest of Group2 happened to be d not c
            if(c>a):                                # ......
                return(routeC,routeD)               # ......
            else: 
                if(b>d):
                    return(routeA,routeB)
                else:
                    return(routeA,routeD)
    else:                                       # Same logic as above but the greatest of Group1 happened to be b not a
        if(c>d):                                  # ......
            if(d>b):                              # ......
                return(routeC,routeD) 
            else: 
                if(a>c): 
                    return(routeA,routeB) 
                else:
                    return(routeB,routeC) 
        else:                                     # Same logic as above but the greatest of Group2 happened to be d not c
            if(c>b):                                # ......
                return(routeC,routeD)               # ......
            else:
                if(a>d):
                    return(routeA,routeB)
                else:
                    return(routeB,routeD)

In [18]:
# conducts algortihm by dividing the population
# in half, each half utilizing a diferent mechanism
# for optimization
def ACO_GA_Hybrid(mapSize, popSize, maxIterations, distanceTable):
    # Step 1 - Initialization
    # optimal ACO settings drawn from experiment 2
    evaporationRate = 0.3
    alpha = 5
    beta = 1
    pheramoneUpdateFunction = AS_PheramoneUpdate
    
    # optimal GA settings drawn from experiment 1
    FitnessFunction = RankBasedSampling
    elitePercentage = 0.1
    mutationProb = 0.006
    
    # algorithm settings
    pheramoneTable = [[0.0000000001 for col in range(mapSize)] for row in range(mapSize)]
    GBestScore_PerItreation = []
    GBest = 0
    
    for iteration in range(maxIterations):
        # Step 2 - Run ACO
        routes = []
        scores = []
        
        for member in range(popSize):
            route = Ant(random.choice(range(mapSize)), mapSize, distanceTable, pheramoneTable, alpha, beta)
            routes += [route]
            scores += [routeScore(route, distanceTable)]
        
        
        
        # Step 3 - Extract Chromosomes from Ants
        # chromosomes = routes
        # evaluationScores = scores
        
        
        # Step 4 - run GA using those genes
        matingPool = FitnessFunction(routes, scores, popSize, mapSize)
        eliteSize = round(popSize * elitePercentage)
        
        # breeding
        for i in range(eliteSize, popSize, 2):
            parent1 = matingPool[i]
            parent2 = matingPool[(i+1) % (popSize-1)]
            swapPoint = random.choice(range(1, mapSize-1))
            
            child1 = [0] * swapPoint
            child2 = [0] * swapPoint
            
            # crossover
            for j in range(swapPoint):
                child1[j] = parent1[j]
                child2[j] = parent2[j]
            
            child1 = child1 + [gene for gene in parent2 if gene not in child1]
            child2 = child2 + [gene for gene in parent1 if gene not in child2]
            
            # mutation
            for gene in range(mapSize):
                if(random.random() < mutationProb): # child1 mutation
                    gene2 = gene
                    while (gene2 == gene):
                        gene2 = random.choice(range(mapSize))
                        
                    temp = child1[gene]
                    child1[gene] = child1[gene2]
                    child1[gene2] = temp

                    
                if(random.random() < mutationProb): # child2 mutation
                    gene2 = gene
                    while (gene2 == gene):
                        gene2 = random.choice(range(mapSize))
                        
                    temp = child2[gene]
                    child2[gene] = child2[gene2]
                    child2[gene2] = temp
            
            routes[i], routes[(i+1) % (popSize-1)] = SteadyStateFunction(parent1, parent2, child1, child2, distanceTable)
        
        
        
        # Step 5 - Update Pheramone table and records
        pheramoneTable = pheramoneUpdateFunction(routes, scores, [i for i in pheramoneTable])
        pheramoneTable = [[(1-evaporationRate) * col for col in row] for row in pheramoneTable]
        
        bestScore = 0
        for i in range(len(routes)):
            score = routeScore(routes[i], distanceTable)
            if(score > bestScore):
                bestScore = score
        
        if(bestScore > GBest):
            GBest = bestScore
        GBestScore_PerItreation += [GBest]
        
    return(GBestScore_PerItreation)
    

In [12]:
# Experiment 4 collecting data from hybrid to solve
# the final Research Question

popSize = 50            # Number of chromosomes to use
maxIterations = 300    # Maximum number of generations

# Traveling Salesman Problem Settings
mapSize = 50        # cities per map
mapCounter = 0
f = open(f"C:/Users/C14460702/Dissertation/Data/Results/Experiment4/Size - {mapSize}/ACO_GA_Hybrid.txt", 'w')
while (True):
    try:
        # get map
        distanceTable = getMap(mapSize,mapCounter)
        data = ACO_GA_Hybrid(mapSize, popSize, maxIterations, distanceTable)
        f.writelines(str(data))
        
        print(f"Finished map {mapCounter+1}!")
        mapCounter += 1
    except FileNotFoundError:
        break
f.close()

Finished map 1!
Finished map 2!
Finished map 3!
Finished map 4!
Finished map 5!
Finished map 6!
Finished map 7!
Finished map 8!
Finished map 9!
Finished map 10!
Finished map 11!
Finished map 12!
Finished map 13!
Finished map 14!
Finished map 15!
Finished map 16!
Finished map 17!
Finished map 18!
Finished map 19!
Finished map 20!
Finished map 21!
Finished map 22!
Finished map 23!
Finished map 24!
Finished map 25!
Finished map 26!
Finished map 27!
Finished map 28!
Finished map 29!
Finished map 30!
Finished map 31!
Finished map 32!
Finished map 33!
Finished map 34!
Finished map 35!
Finished map 36!
Finished map 37!
Finished map 38!
Finished map 39!
Finished map 40!
Finished map 41!
Finished map 42!
Finished map 43!
Finished map 44!
Finished map 45!
Finished map 46!
Finished map 47!
Finished map 48!
Finished map 49!
Finished map 50!
Finished map 51!
Finished map 52!
Finished map 53!
Finished map 54!
Finished map 55!
Finished map 56!
Finished map 57!
Finished map 58!
Finished map 59!
Finish